# Submitting quantum hardware experiments with qSDK and QEMIST Cloud

qSDK allows users to express quantum circuits in various formats, which can be then submitted to one of the quantum cloud services providers using their account credientials and the API. But managing accounts and billing to multiple quantum cloud services in order to access all the devices and services they offer can be cumbersome or complicated depending on your situation.

Users who have a subscription to our QEMIST Cloud platform are able to use qSDK to run quantum hardware experiments on any of the quantum cloud services we support (including [Amazon Braket](https://aws.amazon.com/braket/), simply using their QEMIST Cloud account and credits. 

This short tutorial shows how users can achieve that by simply installing both the qSDK and QEMIST Cloud's client library, and access all the backends available in Amazon Braket.

## Installation & environment

In order to succesfully submit an experiment, you will first need to:
- install qSDK
- install the QEMIST Cloud client library, `qemist-client`.

You then need to set up the two environment variables `QEMIST_PROJECT_ID` and `QEMIST_AUTH_TOKEN`, which are provided to you once you obtained a subscription to QEMIST Cloud. You may set these variables in your OS / terminal, or provide them in your script using the `os` module, as below:

In [1]:
import os

os.environ['QEMIST_PROJECT_ID'] = "b2c5349b-0dce-463c-b026-1fff15097a11"
os.environ['QEMIST_AUTH_TOKEN'] = "v1.McYNGvBE69bYGtZ1XYHxXPCevmonOSUjAACHOoN4tUZfqJ1sTFT9R1q_p7Kst-XTFW8X910g1DnjM5z22J7NBe4"

If these two variables are not properly set, you will get errors later on in some import statements related to `qemist-client`.

## A simple example

First, we define a quantum circuit using the API provided by qSDK. For the purpose of this tutorial, we here explicitly define a short circuit that simply prepares a Bell state. However, this feature works with any quantum circuit expressed in the qSDK format, including those you may have obtained from a complicated custom workflow expressed with qSDK.

In [2]:
from agnostic_simulator import Gate, Circuit

circuit = Circuit([Gate("H", 0), Gate("CNOT", 1, control=0)])
print(circuit)

Circuit object. Size 2 

H         target : 0   
CNOT      target : 1   control : 0   



In order to submit a quantum job and run this circuit on one of the avilable devices, we import a few convenience functions from the qSDK, which rely on the QEMIST client library. We illustrate their usage in the rest of the notebook.

In [3]:
from agnostic_simulator.qpu_connection import job_submit, job_status, job_cancel, job_result

### Job submission

The `job_submit` function can be used to submit a quantum job on the target backends available in Amazon Braket (through their [device arn](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html)), and the number of shots required. Both parameters are required. It returns a ID, used as a handle to your quantum job. This call is asynchronous / non-blocking, which means that it returns without waiting for the job to complete, allowing you to continue with your script in the meantime.

Feel free to uncomment any of the lines below, if you wish to run the following commands on the backend of your choice. Note that the quantum devices are not guaranteed to be immediately available and that you will require a subscription to QEMIST Cloud.

In [4]:
backend = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'
# backend = 'arn:aws:braket:::device/qpu/ionq/ionQdevice'
# backend = 'arn:aws:braket:::device/qpu/rigetti/Aspen-9'
# backend = 'arn:aws:braket:::device/quantum-simulator/amazon/dm1'

In [5]:
from agnostic_simulator.qpu_connection import job_submit, job_status, job_cancel, job_result

job_id = job_submit(circuit, n_shots=10, backend=backend)
print(f"Job submitted with job id :: {job_id}")

{'shots': 10, 'backend': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'}
Job submitted with job id :: 66040883358315195


### Job status

Once your job has been submitted, you may query his status through the `job_status`. It may take time for this job to start, as your request has been queued with the desired cloud services: it could be instant or take hours for the job to start.

The return values are: `ready`, `in_progress`, `complete`, `cancelled`.

In [6]:
print(job_status(job_id))

ready


### Job cancel

A job that has been submitted and has not yet started may be cancelled through the `job_cancel` command, which takes the job ID as parameter. You are free to uncomment this line and try, if you'd like.

In [7]:
# print(job_cancel(job_id))

{'66040883358315195': {'n_children': 0, 'status': 'cancelled'}}


### Job result

Finally, you can make a blocking call to retrieve your results using `job_result`. This function returns the histogram of frequencies associated with your quantum circuit, as well as extra raw data provided by Braket, as nested dictionaries. You are free to investigate and parse the output as you see fit:

In [8]:
output = job_result(job_id)
print(output)

{'fragment_type': 'QuantumCircuit', 'problem_handle': 66040883358315195, 'result': {'results': {'measurements': [[1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [0, 0], [0, 0], [1, 1]], 'measured_qubits': [0, 1], 'measurement_counts': {'00': 4, '11': 6}, 'measurement_probabilities': {'00': 0.4, '11': 0.6}}, 'metadata': {'action': {'results': [], 'instructions': [{'type': 'h', 'target': 0}, {'type': 'cnot', 'target': 1, 'control': 0}], 'basis_rotation_instructions': []}, 'rigettiMetadata': None}}, 'solve_time': 777, 'status': 'complete'}


In [9]:
freqs = output['result']['results']['measurement_probabilities']
counts = output['result']['results']['measurement_counts']
print(f"Frequencies :: {freqs}")
print(f"Counts :: {counts}")

Frequencies :: {'00': 0.4, '11': 0.6}
Counts :: {'00': 4, '11': 6}


### Closing words

This notebook briefly shown how users with a subscription to QEMIST Cloud can benefit from using our wrappers and submit their hardware experiments to a broad range of backends, while not requiring them to manage one or multiple accounts for quantum cloud service providers.